importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import warnings
warnings.filterwarnings("ignore")

importing dataset

In [ ]:
train_data = pd.read_csv(r"C:\\Users\\DELL\Desktop\Dataset\rossmann-store-sales\train.csv", sep=',', parse_dates = True, index_col = 'Date',
                    dtype={'StateHoliday':str, 'SchoolHoliday':str})

In [ ]:
store_data = pd.read_csv(r"C:\\Users\\DELL\Desktop\Dataset\rossmann-store-sales\store.csv")

In [ ]:
print("Number of observations & Number of features", train_data.shape)
train_data.head()

In [ ]:
print("Number of observations & Number of features", store_data.shape)
store_data.head()

In [ ]:
print("Train:\n\n", train_data.isnull().sum(), 
     "\n\nstore:\n\n", store_data.isnull().sum())

In [ ]:
store_data = store_data.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear','Promo2SinceWeek',
                     'Promo2SinceYear', 'PromoInterval'], axis=1)

In [ ]:
sns.distplot(store_data.CompetitionDistance.dropna())
plt.title("Distribution of Store Competition Distance")

In [ ]:
store_data.CompetitionDistance.fillna(store_data.CompetitionDistance.median(), inplace=True)

In [ ]:
train_data['Year'] = train_data.index.year
train_data['Month'] = train_data.index.month
train_data['Day'] = train_data.index.day
train_data['WeekOfYear'] = train_data.index.weekofyear
train_data = train_data.reset_index()

In [ ]:
full_data = pd.merge(train_data, store_data, how='left', on='Store')
full_data.head()

In [ ]:
set(full_data.StateHoliday), set(full_data.StoreType), set(full_data.Assortment)

full_data.loc[full_data.StateHoliday == 0,'StateHoliday'] = full_data.loc[full_data.StateHoliday == 0,'StateHoliday'].astype(str)

In [ ]:
full_data.head()

In [ ]:
sales = full_data[['Year','Month','Store','Sales']].groupby(['Year','Month','Store']).mean()
sales = sales.rename(columns={'Sales':'AvgSales'})
sales = sales.reset_index()

In [ ]:
full_data['sales_key']=full_data['Year'].map(str) + full_data['Month'].map(str) + full_data['Store'].map(str)
sales['sales_key']=sales['Year'].map(str) + sales['Month'].map(str) + sales['Store'].map(str)

cust = full_data[['Year','Month','Store','Customers']].groupby(['Year','Month', 'Store']).mean()
cust = cust.rename(columns={'Customers':'AvgCustomer'})
cust = cust.reset_index()

full_data['cust_key']=full_data['Year'].map(str) + full_data['Month'].map(str) + full_data['Store'].map(str)
cust['cust_key']=cust['Year'].map(str) + cust['Month'].map(str) + cust['Store'].map(str)

In [ ]:
full_data.head()

In [ ]:
full_data = full_data.drop('Customers', axis=1)# drop extra columns
cust = cust.drop(['Year', 'Month', 'Store'], axis=1)

full_data = pd.merge(full_data, cust, how="left", on=('cust_key'))

In [ ]:
full_data.head()

In [ ]:
full_data['StateHoliday'] = full_data.StateHoliday.map({'0':0, 'a':1 ,'b' : 1,'c': 1})

full_data = full_data.drop(['cust_key','sales_key','Store','Date'], axis=1)

full_data_v = full_data.sample(n=5000, random_state=1)


In [ ]:

corr = full_data.corr()

In [ ]:
mask = np.zeros_like(corr, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize = (11, 9))
sns.heatmap(corr, mask = mask,
            square = True, linewidths = .7, ax = ax, cmap = "BuPu")
plt.title("Correlation Heatmap", fontsize=10)

In [ ]:
dfd = full_data

X = dfd.drop('Sales', axis=1) #has everything except sales
Y = dfd.Sales #has only sales

xd = X.copy()
xd = dfd.copy()
xd = pd.get_dummies(xd)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
xd.head()

In [ ]:
xl = X.copy()
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder() # used to normalize labels convert string to numerical data
xl.StateHoliday = label.fit_transform(xl.StateHoliday)
xl.Assortment = label.fit_transform(xl.Assortment)
xl.StoreType = label.fit_transform(xl.StoreType)

In [ ]:
xl.head() #except sales

In [ ]:
from sklearn.model_selection import train_test_split
xd_train,xd_test,yd_train,yd_test = train_test_split(xd,Y,test_size=0.20, random_state=1) 
xl_train,xl_test,yl_train,yl_test = train_test_split(xl,Y,test_size=0.20, random_state=1) 
from sklearn.linear_model import LinearRegression
lin= LinearRegression()
linreg = lin.fit(xd_train, yd_train)

In [ ]:
xl_train.head()

In [ ]:
xd_train.head()

In [ ]:
xd_test.head()

In [ ]:
yd_train.head()

In [ ]:
yl_train.head()

In [ ]:
yd_test.head()

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(x, y):
    return sqrt(mean_squared_error(x, y))
# definte MAPE function
def mape(x, y):
    return np.mean(np.fabs((x-y)/x))*100 

In [ ]:
# get cross validation scores
yd_predicted = linreg.predict(xd_train)
yd_test_predicted = linreg.predict(xd_test)
print("Regression Model Score" , ":" , linreg.score(xd_train, yd_train) , "," ,
      "Out of Sample Test Score" ,":" , linreg.score(xd_test, yd_test))
print("Training RMSE", ":", rmse(yd_train, yd_predicted),
      "Testing RMSE", ":", rmse(yd_test, yd_test_predicted))
print("Training MAPE", ":", mape(yd_train, yd_predicted),
      "Testing MAPE", ":", mape(yd_test, yd_test_predicted))

In [ ]:
# The coefficients
print('Coefficients: \n', lin.coef_)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 50)
knnreg = knn.fit(xd_train, yd_train)

In [ ]:
print("Regression Model Score" , ":" , knnreg.score(xd_train, yd_train) , "," ,
      "Out of Sample Test Score" ,":" , knnreg.score(xd_test, yd_test))
yd_predicted = knnreg.predict(xd_train)
yd_test_predicted = knnreg.predict(xd_test)
print("Training RMSE", ":", rmse(yd_train, yd_predicted),
      "Testing RMSE", ":", rmse(yd_test, yd_test_predicted))
print("Training MAPE", ":", mape(yd_train, yd_predicted),
      "Testing MAPE", ":", mape(yd_test, yd_test_predicted))

In [ ]:
for x in range(1,10):
    knn = KNeighborsRegressor(n_neighbors = x)
    knnreg = knn.fit(xd_train, yd_train)
    print("Regresion Model Score" , ":" , knnreg.score(xd_train, yd_train) , "," ,
      "Out of Sample Test Score" ,":" , knnreg.score(xd_test, yd_test))

inspecting dataset


In [ ]:
print(train_data.describe())

In [ ]:
train_data.StateHoliday.unique()

In [ ]:
store_data = train_data[train_data.Store==150].sort_values('Date')
plt.figure(figsize=(30, 10))  
plt.plot(store_data.Sales.values[:730])

In [ ]:
print("Mean   - {:8,.2f}".format(np.mean(train_data.Sales)))


In [ ]:
maxsales_plot = train_data.groupby("DayOfWeek").sum().plot(kind='bar')
maxsales_plot.set_xlabel("Day of the week")
maxsales_plot.set_ylabel("Units")

In [ ]:
df_items = train_data[["DayOfWeek", "Sales"]]
df_items.groupby("DayOfWeek").sum().plot(kind="bar")

In [ ]:
plt.plot( 'Sales', 'Promo', data=train_data, linestyle='none', marker='o')
plt.xlabel('Sales')
plt.ylabel('Promo')
plt.show()

In [ ]:
sns.distplot(train_data.Customers.dropna())
plt.title("Distribution of Customers")

In [ ]:
sns.distplot(train_data.Sales.dropna())
plt.title("Distribution of Sales")

In [ ]:
plt.plot( 'Sales', 'Date', data=train_data, linestyle='none', marker='o')
plt.xlabel('Sales')
plt.ylabel('Date')
plt.show()

In [ ]:
new_plot = train_data[train_data.Customers==500].sort_values('Date')
plt.figure(figsize=(15, 5))  
plt.plot(new_plot.Sales.values[:730])


In [ ]:
# The coefficients
print('Coefficients: \n', lin.coef_)

end

In [ ]:
sns.regplot(x=train_data["Sales"], y=train_data["Promo"], line_kws={"color":"r","alpha":0.7,"lw":5})
#sns.plt.show()

In [ ]:
sns.set(color_codes=True)


In [ ]:
sns.pairplot(train_data, kind="reg",vars=["Sales","Customers"])
plt.show()

In [ ]:
sns.pairplot(train_data, kind="reg",vars=["Promo","Sales"])
plt.show()

In [ ]:
sns.pairplot(train_data, kind="scatter",vars=["Sales","Customers"])